In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lumnisai
from lumnisai import Scope, ApiProvider, AsyncClient
from IPython.display import Markdown
from tqdm import tqdm
import asyncio
from dotenv import load_dotenv
import os

load_dotenv()


def display_markdown(text):
    display(Markdown(text))

# Create client
lumnisai_client = lumnisai.AsyncClient()

# List users
users = await lumnisai_client.list_users(page_size=50)

# Check if alice@test-acme.one exists in users
alice_user = next((user for user in users.users if user.email == "alice@test-acme.one"), None)

# Create user if not found
if alice_user is None:
    user = await lumnisai_client.create_user( email="alice@test-acme.one", first_name="Alice", last_name="Doe" )

# Print users
print("Existing Users:")
for user in users.users:
    print("    ", user.email)

print("="*50)

# List API keys
api_keys = await lumnisai_client.list_api_keys()
print("Existing API keys:")
for api_key in api_keys:
    print("    ", api_key.provider, "Exists - Created at", api_key.created_at)

print("="*50)

# Add API keys
await lumnisai_client.add_api_key(ApiProvider.OPENAI_API_KEY, os.getenv("OPENAI_API_KEY"))
await lumnisai_client.add_api_key(ApiProvider.EXA_API_KEY, os.getenv("EXA_API_KEY"))

print("Setup Complete!")

Existing Users:
     alice@test-acme.one
Existing API keys:
     EXA_API_KEY Exists - Created at 2025-06-20T02:50:30.419402+00:00
     OPENAI_API_KEY Exists - Created at 2025-06-20T02:50:25.676567+00:00
Setup Complete!


In [ ]:
user_scoped_client = lumnisai.AsyncClient().for_user("alice@test-acme.one")
response = await user_scoped_client.invoke(
    prompt="What is the weather in Tokyo?",
    progress=True
)
# print(response.progress)
display_markdown(response.output_text)

In [ ]:
response = await user_scoped_client.invoke(
    prompt="What are the latest trends in AI agents and machine learning? ",
    progress=True
    )
# print(response.progress)
display_markdown(response.output_text)

NameError: name 'user_scoped_client' is not defined

In [3]:
"""
Streaming responses example using tenant scope (default).
Shows real-time progress updates using invoke(stream=True).
"""

# Auto-initializes on first use - no context manager needed  
client = lumnisai.AsyncClient(scope=Scope.TENANT)

print("Starting research task...")

async for update in await client.invoke(
    "What is the current weather in Tokyo and boston? What is the latest news in AI and machine learning?",
    stream=True,
    poll_interval=4,
    wait_timeout=10
):
        # Show progress messages if available
        if update.progress and len(update.progress) > 0:
            latest = update.progress[-1]
            print(f"{latest.state.upper()}: {latest.message}")
        
        # Show final result
        if update.status == "succeeded" and update.output_text:
            print("\n" + "="*50)
            print("Task completed!")
        elif update.status == "failed":
            print("Task failed!")
            break



Starting research task...
Status: in_progress
PROCESSING: I'm now searching for the current weather in Tokyo and Boston, as well as the latest news in AI and machine learning. I'll gather this information and then summarize it for you.
Status: in_progress
PROCESSING: I'm now analyzing all the search results to prepare a clear, organized summary of the current weather in Tokyo and Boston, as well as the latest news in AI and machine learning. I'll include sources for each section.
Status: succeeded
COMPLETED: Response completed successfully.

Task completed!


In [4]:
display_markdown(update.output_text)

Certainly! Here’s a clear, organized summary of the latest information on the current weather in Tokyo and Boston, as well as the latest news in AI and machine learning. Each section includes a list of sources in markdown format.

---

## 1. Current Weather in Tokyo

**Summary (as of June 15, 2025):**

- **Temperature:** Highs around 86°F (30°C), lows near 72°F (22°C)
- **Conditions:** Showery and cloudy, with high humidity (up to 98% reported in the morning)
- **Forecast:** The coming week is expected to be record-breakingly hot, with several days above 30°C and possible "mousho" (extreme heat) days above 35°C, especially June 17–18. Nights will remain warm, raising the risk of heat-related illnesses.
- **Special Notes:** The Tokyo area is experiencing a stretch of “梅雨の晴れ間” (clear breaks during the rainy season), contributing to the unusual heat. Residents are advised to take precautions against heatstroke, especially at night.

**Sources:**
- [Time and Date: Tokyo Weather](https://www.timeanddate.com/weather/japan/tokyo)
- [tenki.jp Weather News (Japanese)](https://tenki.jp/forecaster/r_fukutomi/2025/06/15/34131.html)
- [BBC Weather: Tokyo](https://www.bbc.com/weather/1850147)
- [Weather.com: Tokyo Forecast](https://weather.com/weather/today/l/Minato+ku+Tokyo+Prefecture+Japan?canonicalCityId=89a9a327ec5fd290c4d12f51a20485cb)
- [The Weather Network: Tokyo](https://www.theweathernetwork.com/en/city/jp/tokyo/tokyo/current)

---

## 2. Current Weather in Boston

**Summary (as of June 16, 2025):**

- **Temperature:** Around 66°F (19°C) in the evening; highs earlier in the day reached 79°F (26°C)
- **Conditions:** Mostly cloudy to partly cloudy; humidity around 54–61%
- **Forecast:** Tonight will be partly cloudy with lows near 59°F (15°C). Tomorrow (June 17) is expected to be much warmer, with highs reaching 82°F (28°C) and continued partly to mostly cloudy skies.
- **Special Notes:** The week ahead will see a steady climb in temperatures, with peak heat and humidity expected later in the week. Occasional storms are possible as a cold front approaches.

**Sources:**
- [Weather.com: Boston, MA](https://weather.com/weather/today/l/USMA0046:1:US)
- [Wunderground: Boston, MA](https://www.wunderground.com/weather/us/ma/boston)
- [NBC Boston Weather](https://www.nbcboston.com/weather/)
- [WHDH Weather Blog](https://whdh.com/weather-blog/cool-fathers-day-forecast/)
- [WCVB Channel 5 Weather](https://www.wcvb.com/weather)

---

## 3. Latest News in AI and Machine Learning

**Summary (June 2025):**

- **Major Breakthroughs:**
    - **Brain-inspired AI:** Researchers developed "Lp-Convolution," a new AI technique that makes computer vision more efficient and brain-like, improving accuracy and robustness in image recognition while reducing computational costs.
    - **AI for Medical Diagnosis:** A new ultra-lightweight AI model can diagnose lung cancer using just a laptop, outperforming larger models and making advanced diagnostics accessible even with small datasets.
    - **AI Discovers New Math:** Google DeepMind’s AlphaEvolve agent improved on Strassen’s matrix multiplication algorithm for the first time in over 50 years, showcasing AI’s potential for scientific discovery.
    - **Generative AI in Biology:** AI-designed DNA sequences can now control gene expression in healthy mammalian cells, opening doors for precise gene therapies.
    - **Text-to-Video Advances:** New models like MagicTime can generate realistic, physics-aware metamorphic videos (e.g., a flower blooming), moving closer to simulating real-world processes.
- **Industry & Policy:**
    - **Apple, Google, Meta, and others** are rolling out new AI-powered features in consumer devices and services.
    - **OpenAI, Anthropic, and DeepSeek** continue to release new models, with OpenAI’s o3-mini and Google’s Gemini 3 among the latest.
    - **AI Safety & Regulation:** The US and international partners are convening to address AI safety, while the UK has criminalized AI-generated child abuse material.
    - **AI in the Newsroom & Content:** Media organizations are integrating AI for content creation and moderation, with some layoffs attributed to automation.
    - **AI in Warfare:** Ukraine and other countries are deploying AI-powered drones and weapon detection systems.
    - **AI in Healthcare:** New AI tests and tools are being developed for cancer diagnostics, personalized medicine, and digital therapeutics.
    - **AI Market Growth:** The global AI market is projected to reach $4.8 trillion by 2033.

**Sources:**
- [ScienceDaily: Brain-inspired AI breakthrough](https://www.sciencedaily.com/releases/2025/04/250422131924.htm)
- [News Medical: AI model diagnoses lung cancer using just a laptop](https://www.news-medical.net/news/20250605/AI-model-diagnoses-lung-cancer-using-just-a-laptop.aspx)
- [LinkedIn: Google's AlphaEvolve math breakthrough](https://www.linkedin.com/pulse/ai-math-breakthrough-googles-alphaevolve-improves-strassens-borish-umgdc)
- [Crescendo AI: Latest AI News](https://www.crescendo.ai/news/latest-ai-news-and-updates)
- [TechXplore: AI & ML News](https://techxplore.com/machine-learning-ai-news/)
- [MIT Sloan: Machine learning and generative AI in 2025](https://mitsloan.mit.edu/ideas-made-to-matter/machine-learning-and-generative-ai-what-are-they-good-for)
- [ScienceDaily: AI-designed DNA controls genes](https://www.sciencedaily.com/releases/2025/05/250508112324.htm)
- [ScienceDaily: Text-to-video AI blossoms](https://www.sciencedaily.com/releases/2025/05/250505170633.htm)
- [Reuters: OpenAI valued at $340B](https://www.reuters.com/technology/artificial-intelligence/openai-talks-investment-round-valuing-it-up-340-billion-wsj-reports-2025-01-30/)
- [OpenAI: 12 Days of OpenAI](https://openai.com/12-days/)
- [Google AI Blog: Gemini and Veo updates](https://blog.google/technology/google-deepmind/google-gemini-ai-update-december-2024)

---

If you need more details on any topic, let me know!